<a href="https://colab.research.google.com/github/mahmed31098/Improving-Hate-Speech-Detection-Using-Pragmatic-Level-Information/blob/main/unsloth_mistral_7b_or_llama2_7b_bnb_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

To install Unsloth on your own computer, follow the installation instructions on unsloth's Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save).

In [4]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

* Unsloth supports 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since they do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), they support downloading 4bit models **4x faster**! [Their repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # Choose ANY! eg unsloth/llama-2-7b-bnb-4bit
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [mahmed31](https://huggingface.co/datasets/mahmed31/revised_Toraman22_hate_speech_v2), which is a filtered version of 68K of the original [Toraman et al., LREC 2022](https://aclanthology.org/2022.lrec-1.238/).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

In [ ]:
import re
import requests
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("mahmed31/revised_Toraman22_hate_speech_v2")
df = pd.DataFrame(dataset['train'])

# domain_mapping = {0: 'Religion', 1: 'Gender', 2: 'Race', 3: 'Politics', 4: 'Sports'}
label_mapping = {0: 'Normal', 1: 'Offensive', 2: 'Hate'}

# Selecting 5 samples for each class
class_0_indices = df[df['label'] == 0].index[:1615]
class_1_indices = df[df['label'] == 1].index[:1615]
class_2_indices = df[df['label'] == 2].index[:1615]

df['label'] = df['label'].map(label_mapping)

# Combining indices of all classes
selected_indices = np.concatenate([class_0_indices, class_1_indices, class_2_indices])

X = df.iloc[selected_indices]['text'].values
y = df.iloc[selected_indices]['label'].values

# Splitting dataset for demonstration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

data_dict = {
    'X_train': X_train,
    'y_train': y_train,
}

# Convert the dictionary into a Dataset object
formatted_dataset = Dataset.from_dict(data_dict)

test_data_dict = {
    'X_test': X_test,
    'y_test': y_test,
}

# Convert the dictionary into a Dataset object
formatted_test_dataset = Dataset.from_dict(test_data_dict)


<ipython-input-4-75f9049f4f64>:14: DtypeWarning: Columns (0,4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Toraman22_hate_speech_v2.tsv', delimiter='\t', encoding='utf-8').dropna()


In [ ]:
len(X_train)

3876

In [ ]:
# Use the prompt below to apply zero shot prompting which doesn't implement pragmatic level information

prompt = """Based on the following text content, identify the category of the text from categories_list: ['Hate', 'Offensive', 'Normal']
If the text does not fit into any of the above mentioned categories. Please return the answer nearest to any of the categories mentioned in categories_list.
Return only one category from the categories_list above.
### text:
{}

### category:
{}"""



'''
# Use the prompt below to apply few shot prompting which implements pragmatic level information

prompt = """Based on the following text content, identify the category of the text from categories_list: ['Hate', 'Offensive', 'Normal'].
For your reference, definitions of the labels in the categories_list are as follows:
- 'Hate': Directly targets or incites harm against specific groups or individuals.
- 'Offensive': Contains insults or derogatory language but does not incite direct harm.
- 'Normal': Does not contain offensive content or hate speech.

Consider the following examples to guide your analysis:
1. 'I can’t live in a world where gay marriage is legal.' Okay, so die. - categories_list: Hate,
2. 'Good night to the Turks, death to the Americans.' - categories_list: Hate,
3. 'Self-proclaimed atheist doesn’t make you a cool kid, bitch.' - categories_list: Offensive,
4. 'After that, the referee, who does not count and apply the 6-second rule, will be dishonest.' - categories_list: Offensive,
5. 'Biden, you're a liar and a cheat and an old idiot.' - categories_list: Offensive,

If the text does not fit into any of the above mentioned categories. Please return the answer nearest to any of the categories mentioned in categories_list.
Return only one category from the categories_list above.
### text:
{}

### category:
{}"""
'''



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    text = examples["X_train"]
    category     = examples["y_train"]
    #categories_list = examples["categories_list"]
    contents = []
    for X_train, y_train in zip(text, category):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        content = prompt.format(X_train, y_train) + EOS_TOKEN
        contents.append(content)
    return { "response" : contents, }
pass

dataset = formatted_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "response",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/3876 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.5 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,876 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.926400
2,3.031000
3,2.830200
4,2.266500
5,2.189700
6,1.653200
7,1.304100
8,1.116700
9,1.201600
10,0.979100


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

343.7275 seconds used for training.
5.73 minutes used for training.
Peak reserved memory = 5.16 GB.
Peak reserved memory for training = 0.66 GB.
Peak reserved memory % of max memory = 34.988 %.
Peak reserved memory for training % of max memory = 4.475 %.


In [ ]:
model.save_pretrained("lora_model")

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [12]:
if False:
    from unsloth import FastLanguageModel
    import torch
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# prompt = You MUST copy from above! (like the way I did below)

prompt = """Based on the following text content, identify the category of the text from categories_list: ['Hate', 'Offensive', 'Normal']
If the text does not fit into any of the above mentioned categories. Please return the answer nearest to any of the categories mentioned in categories_list.
Return only one category from the categories_list above.
### text:
{}

### category:
{}"""

inputs = tokenizer(
[
    prompt.format(
        "You seems like a nice person", # tweet or text given as input
        "", # will predict the category
    ),
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
decoded_outputs = tokenizer.batch_decode(outputs)
category = decoded_outputs[0].split("### category:")[1].strip().split("\n")[0]
category

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Normal'